## Save activation values

In [11]:
import sys

sys.path.append('donkeycar')
sys.path.append('src')

%cd ..

/Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis


In [12]:
from tensorflow import keras
import multiprocessing
import numpy as np
from losses import get_pipe, get_dataset_from_pipe
from donkeycar.utils import get_model_by_type
from dataloader import load_records
import donkeycar as dk


In [14]:
cfg = dk.load_config(config_path='mycar/config.py')

# None means all the data is used.
# use a smaller size, like 200 for testing end to end.
TUBRECORD_SIZE = None

cfg.SEQUENCE_LENGTH = 3
cfg.WANDB_ENABLED = False
cfg.TRANSFORMATIONS = ['CROP']
cfg.ROI_CROP_TOP = 60
cfg.ROI_CROP_KEEP_SIZE = False
N_FOLDS = 5
DIRECTION = 'CC'
DATA_PATH = 'data'

model_type = '3d-modified-steering'

loading config file: mycar/config.py
loading personal config over-rides from myconfig.py


In [15]:
fast_speed_data = load_records('faster-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(fast_speed_data)


INFO:donkeycar.pipeline.types:Loading only the first tub from paths ['data/faster-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/faster-autodrive/catalog_20.catalog


19250

In [16]:
slow_speed_data = load_records('slower-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(slow_speed_data)


INFO:donkeycar.pipeline.types:Loading only the first tub from paths ['data/slower-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/slower-autodrive/catalog_20.catalog


20304

In [17]:
get_model_by_type(model_type, cfg).interpreter.model.summary()


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3


Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_in (InputLayer)         [(None, 3, 60, 160, 3)]   0         
                                                                 
 conv3d_12 (Conv3D)          (None, 1, 20, 53, 16)     1312      
                                                                 
 max_pooling3d_8 (MaxPooling  (None, 1, 10, 26, 16)    0         
 3D)                                                             
                                                                 
 conv3d_13 (Conv3D)          (None, 1, 10, 26, 32)     4640      
                                                                 
 max_pooling3d_9 (MaxPooling  (None, 1, 5, 13, 32)     0         
 3D)                                                             
                                                                 
 conv3d_14 (Conv3D)          (None, 1, 5, 13, 32)      9248  

In [18]:
from dataloader import get_fold_indices
from model_loader import load_model
import math

fast_activations = np.zeros((len(fast_speed_data), 256))

MODELS_PATH = 'models/fast-multi-frame'

for fold, (train_index, test_index) in enumerate(get_fold_indices(fast_speed_data), start=1):
    model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    activations_model = keras.Model(model.interpreter.model.input, model.interpreter.model.get_layer('activation_1').output)

    test_records = [fast_speed_data[i] for i in test_index]
    cfg.BATCH_SIZE = max((math.gcd(len(test_records), i) for i in range(2, 400)))
    pipe = get_pipe(model, cfg, test_records)
    data = get_dataset_from_pipe(pipe)
    fold_test_activations = activations_model.predict(
        data,
        workers=multiprocessing.cpu_count(),
        use_multiprocessing=True,
        steps=len(pipe),
        verbose=1)
    fast_activations[test_index, :] = fold_test_activations

In [ ]:
np.save("ood/nighttime_ai_fast_model_fast_data_activations.npy", np.float16(fast_activations))


In [23]:
!curl -L "https://www.dropbox.com/s/wodx2qhm8t3v2gy/fast-multi-frame-no-folds.zip?dl=1" --output models/fast-multi-frame-no-folds.zip && \
curl -L "https://www.dropbox.com/s/fiexl1ke4nghg1n/slow-multi-frame-no-folds.zip?dl=1" --output models/slow-multi-frame-no-folds.zip && \
unzip models/fast-multi-frame-no-folds.zip -d models && \
unzip models/slow-multi-frame-no-folds.zip -d models


In [21]:
MODELS_PATH = 'models/fast-multi-frame-no-folds'

model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, 1, 1)
activations_model = keras.Model(model.interpreter.model.input, model.interpreter.model.get_layer('activation_1').output)

cfg.BATCH_SIZE = max((math.gcd(len(slow_speed_data), i) for i in range(2, 400)))
pipe = get_pipe(model, cfg, slow_speed_data)
data = get_dataset_from_pipe(pipe)
slow_activations = activations_model.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)


In [26]:
np.save("ood/nighttime_ai_fast_model_slow_data_activations.npy", np.float16(slow_activations))


In [22]:
slow_activations = np.zeros((len(slow_speed_data), 256))

MODELS_PATH = 'models/slow-multi-frame'

for fold, (train_index, test_index) in enumerate(get_fold_indices(slow_speed_data), start=1):
    model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    activations_model = keras.Model(model.interpreter.model.input, model.interpreter.model.get_layer('activation_1').output)

    test_records = [slow_speed_data[i] for i in test_index]
    cfg.BATCH_SIZE = max((math.gcd(len(test_records), i) for i in range(2, 400)))
    pipe = get_pipe(model, cfg, test_records)
    data = get_dataset_from_pipe(pipe)
    fold_test_activations = activations_model.predict(
        data,
        workers=multiprocessing.cpu_count(),
        use_multiprocessing=True,
        steps=len(pipe),
        verbose=1)
    slow_activations[test_index, :] = fold_test_activations

In [ ]:
np.save("ood/nighttime_ai_slow_model_slow_data_activations.npy", np.float16(slow_activations))

In [23]:

MODELS_PATH = 'models/slow-multi-frame-no-folds'

model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, 1, 1)
activations_model = keras.Model(model.interpreter.model.input, model.interpreter.model.get_layer('activation_1').output)

cfg.BATCH_SIZE = max((math.gcd(len(fast_speed_data), i) for i in range(2, 400)))
pipe = get_pipe(model, cfg, fast_speed_data)
data = get_dataset_from_pipe(pipe)
fast_activations = activations_model.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)


In [28]:
np.save("ood/nighttime_ai_slow_model_fast_data_activations.npy", np.float16(fast_activations))
